In [148]:
import numpy as np
import pandas as pd
from stocktrends import Renko, indicators
import statsmodels.api as sm
import yfinance as yf
import copy

In [149]:
tickers = ["INFY.NS", "TCS.NS", "WIPRO.NS", "TATASTEEL.NS"]
ohlc_intraday = {}
attempt = 0 # initializing passthrough variable
drop = [] # initializing list to store tickers whose close price was successfully extracted

In [150]:
while len(tickers) != 0 and attempt <=5:
    tickers = [j for j in tickers if j not in drop]
    for i in range(len(tickers)):        
        try:
            # ohlc_intraday[tickers[i]] = ts.get_intraday(symbol=tickers[i],interval='5min', outputsize='full')[0]
            ohlc_intraday[tickers[i]] = yf.download(tickers[i],period='60d' , interval='15m')
            # ohlc_intraday[tickers[i]].columns = ["Open","High","Low","Adj Close","Volume"]
            drop.append(tickers[i])      
        except:
            print(tickers[i]," :failed to fetch data...retrying")
            continue
    attempt+=1

 
tickers = ohlc_intraday.keys() # redefine tickers variable after removing any tickers with corrupted data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [151]:
movingDF = copy.deepcopy(ohlc_intraday)
tickers_signal = {}
tickers_ret = {}

In [152]:
#movingDF['INFY.NS']["A"]

In [153]:
movingAverage = []
for ticker in tickers:
    print("merging for ",ticker)
    
    #movingDF[ticker]["50EMA"]= movingDF[ticker]["Adj Close"].ewm(span=50,min_periods=50).mean()
    movingDF[ticker]["44MA"]= movingDF[ticker]["Adj Close"].rolling(30).mean()
    movingDF[ticker].dropna(inplace=True)

    tickers_signal[ticker] = ""
    tickers_ret[ticker] = []

merging for  INFY.NS
merging for  TCS.NS
merging for  WIPRO.NS
merging for  TATASTEEL.NS


In [154]:
movingDF['INFY.NS']['44MA']

Datetime
2022-01-25 10:15:00+05:30    1740.500008
2022-01-25 10:30:00+05:30    1739.486674
2022-01-25 10:45:00+05:30    1738.423340
2022-01-25 11:00:00+05:30    1737.521672
2022-01-25 11:15:00+05:30    1736.701672
                                ...     
2022-04-22 14:30:00+05:30    1605.880001
2022-04-22 14:45:00+05:30    1604.578333
2022-04-22 15:00:00+05:30    1603.386666
2022-04-22 15:15:00+05:30    1602.180001
2022-04-22 15:30:00+05:30    1600.966667
Name: 44MA, Length: 1472, dtype: float64

In [155]:
#Identifying signals and calculating daily returns
for ticker in movingDF:
    print("calculating daily returns for ", ticker)
    for i in range(len(movingDF[ticker])):
        if tickers_signal[ticker] == "":
            if i > 0:
                if movingDF[ticker]["Adj Close"][i] >= movingDF[ticker]["44MA"][i]:
                    tickers_signal[ticker] = "Buy"
                elif movingDF[ticker]["Adj Close"][i] < movingDF[ticker]["44MA"][i]:
                    tickers_signal[ticker] = "Sell"
                
        elif tickers_signal[ticker] == "Buy":
            tickers_ret[ticker].append((movingDF[ticker]["Adj Close"][i]/movingDF[ticker]["Adj Close"][i-1])-1)
            if i > 0:
                if ohlc_renko[ticker]["bar_num"][i]<=-2 and ohlc_renko[ticker]["macd"][i]<ohlc_renko[ticker]["macd_sig"][i] and ohlc_renko[ticker]["macd_slope"][i]<ohlc_renko[ticker]["macd_sig_slope"][i]:
                    tickers_signal[ticker] = "Sell"
                elif ohlc_renko[ticker]["macd"][i]<ohlc_renko[ticker]["macd_sig"][i] and ohlc_renko[ticker]["macd_slope"][i]<ohlc_renko[ticker]["macd_sig_slope"][i]:
                    tickers_signal[ticker] = ""
                
        elif tickers_signal[ticker] == "Sell":
            tickers_ret[ticker].append((ohlc_renko[ticker]["Adj Close"][i-1]/ohlc_renko[ticker]["Adj Close"][i])-1)
            if i > 0:
                if movingDF[ticker]["bar_num"][i]>=2 and movingDF[ticker]["macd"][i]>movingDF[ticker]["macd_sig"][i] and ohlc_renko[ticker]["macd_slope"][i]>ohlc_renko[ticker]["macd_sig_slope"][i]:
                    tickers_signal[ticker] = "Buy"
                elif ohlc_renko[ticker]["macd"][i]>ohlc_renko[ticker]["macd_sig"][i] and ohlc_renko[ticker]["macd_slope"][i]>ohlc_renko[ticker]["macd_sig_slope"][i]:
                    tickers_signal[ticker] = ""
    movingDF[ticker]["ret"] = np.array(tickers_ret[ticker])

calculating daily returns for  INFY.NS


NameError: name 'ohlc_renko' is not defined

In [ ]:
tickers_signal